In [ ]:
!pip install pretty_midi mir_eval numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 15.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.3 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592289 sha256=2f11e825cc738f75eb5266b694719227c4bd95b37ed5281b657d51ca1eb87a91
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
  Created wheel for mir_eval: filename=mir_eval-0.7-py3-none-any.whl size=100704 sha256=9c2a68b7a3a7c58cc27d959775d7317d4406685ea7df52b6aedf653237813b10
  Stored in directory: /root/.cache/pip/wheels/3e/2f/0d/dda9c4c77a170e21356b6afa2f7d9bb078338634ba05d94e3f
Successfully built pretty_midi mir_eval
  Attempting uninstall: packa

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Remove _basic_pitch from midi files
# import os

# def rename_files(folder_path):
#     for filename in os.listdir(folder_path):
#         if '_basic_pitch' in filename:
#             new_filename = filename.replace('_basic_pitch', '')
#             old_file_path = os.path.join(folder_path, filename)
#             new_file_path = os.path.join(folder_path, new_filename)

#             os.rename(old_file_path, new_file_path)
#             print(f"Renamed '{filename}' to '{new_filename}'")

# os.chdir("/content/drive/MyDrive/Dissertation Code")
# folder_path = "OMAPS/evaluation/basic-pitch"
# rename_files(folder_path)


In [ ]:
import os
import pretty_midi
import mir_eval
import numpy as np

os.chdir("/content/drive/MyDrive/Dissertation Code")

# def midi_to_freq(midi):
#     if midi >= 0:
#         return 2 ** ((midi - 69) / 12) * 440
#     else:
#         return 0

# def read_note_data_from_text(file_path):
#     intervals = []
#     pitches = []
#     with open(file_path, 'r') as file:
#         for line in file:
#             start, end, pitch = line.strip().split()
#             intervals.append([float(start), float(end)])
#             pitches.append(int(pitch))
#     # print(np.array(intervals))
#     return np.array(intervals), np.array(pitches)

def read_note_data_from_text(file_path): # For OMAPS2
    """Read note data from a text file."""
    intervals = []
    pitches = []
    velocities = []
    with open(file_path, 'r') as file:
        for line in file:
            columns = line.strip().split()
            # Extract onset, offset, pitch and velocity
            start, end, pitch, velocity = columns
            intervals.append([float(start), float(end)])
            pitches.append(int(pitch))
            velocities.append(int(velocity))
    return np.array(intervals), np.array(pitches)

# def prepare_data_for_evaluation(midi_file, cc_list=[64, 67], split_pedal=False):
#     midi_data = pretty_midi.PrettyMIDI(midi_file)
#     notes = []
#     for instrument in midi_data.instruments:
#         # Filter by piano instruments
#         if instrument.program in range(0, 8):
#             notes.extend(instrument.notes)

#     # Process notes and pedals
#     intervals = []
#     pitches = []
#     velocities = []
#     for note in notes:
#         if note.pitch >= 0 or -note.pitch in cc_list:
#             intervals.append([note.start, note.end])
#             pitches.append(midi_to_freq(note.pitch))
#             velocities.append(note.velocity)

#     intervals = np.array(intervals)
#     pitches = np.array(pitches)
#     velocities = np.array(velocities)

#     return intervals, pitches, velocities

# def evaluate_midi(predicted_midi_file, truth_midi_file):
#     predicted_intervals, predicted_pitches, _ = prepare_data_for_evaluation(predicted_midi_file)
#     truth_intervals, truth_pitches, _ = prepare_data_for_evaluation(truth_midi_file)

#     precision, recall, f1_score, _ = mir_eval.transcription.precision_recall_f1_overlap(
#         truth_intervals, truth_pitches, predicted_intervals, predicted_pitches)

#     return precision, recall, f1_score

def prepare_data_for_evaluation(midi_file):
    midi_data = pretty_midi.PrettyMIDI(midi_file)
    intervals = []
    pitches = []
    for instrument in midi_data.instruments:
        # For non-drum instruments
        for note in instrument.notes:
            intervals.append([note.start, note.end])
            pitches.append(note.pitch)
    # print(np.array(intervals))
    return np.array(intervals), np.array(pitches)

def read_note_data_from_npy(file_path):
    # Load data from the .npy file
    data = np.load(file_path)

    # Find onset, offset and pitch
    intervals = data[:, :2]
    pitches = data[:, 2]

    return intervals, pitches

def evaluate_midi(predicted_midi_file, truth_midi_file):
    predicted_intervals, predicted_pitches = prepare_data_for_evaluation(predicted_midi_file)
    truth_intervals, truth_pitches = read_note_data_from_text(truth_midi_file)

    # print("done")

    # mir_eval.transcription.validate(truth_intervals, truth_pitches, predicted_intervals, predicted_pitches)

    # Full evaluation considering only onset
    full_precision, full_recall, full_f1_score, _ = mir_eval.transcription.precision_recall_f1_overlap(
        truth_intervals, truth_pitches, predicted_intervals, predicted_pitches,
        onset_tolerance=0.05, pitch_tolerance=50.0, offset_ratio=None, offset_min_tolerance=0.05, strict=False, beta=1.0)

    # Onset evaluation
    onset_precision, onset_recall, onset_f1_score = mir_eval.transcription.onset_precision_recall_f1(
        truth_intervals, predicted_intervals, onset_tolerance=0.05, strict=False, beta=1.0)

    # Offset evaluation
    offset_precision, offset_recall, offset_f1_score = mir_eval.transcription.offset_precision_recall_f1(
        truth_intervals, predicted_intervals, offset_ratio=0.2, offset_min_tolerance=0.05, strict=False, beta=1.0)

    absolute_error = mir_eval.alignment.absolute_error(truth_intervals, predicted_intervals)

    return {
        'full': (full_precision, full_recall, full_f1_score),
        'onset': (onset_precision, onset_recall, onset_f1_score),
        'offset': (offset_precision, offset_recall, offset_f1_score).
        'absoulte_error': absolute_error
    }

# def evaluate_folders(predicted_folder, truth_folder, output_file):
#     precisions = []
#     recalls = []
#     f1_scores = []

#     with open(output_file, 'w') as f:
#         for predicted_file in os.listdir(predicted_folder):
#             if not predicted_file.endswith('.mid'):
#                 continue  # Skip files that are not MIDI files
#             predicted_path = os.path.join(predicted_folder, predicted_file)
#             # Change the extension
#             truth_file = predicted_file.rsplit('.', 1)[0] + '.txt'
#             truth_path = os.path.join(truth_folder, truth_file)

#             if os.path.isfile(truth_path):
#                 try:
#                     precision, recall, f1_score = evaluate_midi(predicted_path, truth_path)
#                     precisions.append(precision)
#                     recalls.append(recall)
#                     f1_scores.append(f1_score)

#                     f.write(f"{predicted_file}: Precision={precision}, Recall={recall}, F1 Score={f1_score}\n")
#                 except ValueError as e:
#                     print(f"Error processing {predicted_file}: {e}")
#                     f.write(f"Error processing {predicted_file}: {e}\n")
#             else:
#                 print(f"Ground truth file for {predicted_file} not found")
#                 f.write(f"Ground truth file for {predicted_file} not found\n")

#         if precisions:
#             combined_precision = np.mean(precisions)
#             combined_recall = np.mean(recalls)
#             combined_f1_score = np.mean(f1_scores)

#             # Write the combined metrics to the file
#             f.write(f"\nCombined Precision: {combined_precision}\n")
#             f.write(f"Combined Recall: {combined_recall}\n")
#             f.write(f"Combined F1 Score: {combined_f1_score}\n")
#         else:
#             combined_precision = combined_recall = combined_f1_score = 0
#             f.write("No MIDI files were successfully processed\n")

#     return combined_precision, combined_recall, combined_f1_score

# # Usage example:
# os.chdir("/content/drive/MyDrive/Dissertation Code")

# output_file = "test_evaluation_results.txt"
# predicted_folder = "OMAPS/evaluation/skipping-the-frame-level"
# truth_folder = "OMAPS/complete/text"

# combined_precision, combined_recall, combined_f1_score = evaluate_folders(predicted_folder, truth_folder, output_file)

# print(f"Combined Precision: {combined_precision}")
# print(f"Combined Recall: {combined_recall}")
# print(f"Combined F1 Score: {combined_f1_score}")



def evaluate_folders(predicted_folder, truth_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    full_eval_path = os.path.join(output_folder, "full_evaluation_results.txt")
    onset_eval_path = os.path.join(output_folder, "onset_evaluation_results.txt")
    offset_eval_path = os.path.join(output_folder, "offset_evaluation_results.txt")
    full_avg_path = os.path.join(output_folder, "full_evaluation_average.txt")
    onset_avg_path = os.path.join(output_folder, "onset_evaluation_average.txt")
    offset_avg_path = os.path.join(output_folder, "offset_evaluation_average.txt")

    full_scores = []
    onset_scores = []
    offset_scores = []

    with open(full_eval_path, 'w') as full_file, open(onset_eval_path, 'w') as onset_file, open(offset_eval_path, 'w') as offset_file:
        full_file.write("File, Precision, Recall, F1 Score\n")
        onset_file.write("File, Precision, Recall, F1 Score\n")
        offset_file.write("File, Precision, Recall, F1 Score\n")

    for predicted_file in os.listdir(predicted_folder):
        if not predicted_file.endswith('.mid'):
            continue

        predicted_path = os.path.join(predicted_folder, predicted_file)
        truth_file = predicted_file.rsplit('.', 1)[0] + '.txt'
        truth_path = os.path.join(truth_folder, truth_file)

        if os.path.isfile(truth_path):
            try:
                results = evaluate_midi(predicted_path, truth_path)

                full_scores.append(results['full'])
                onset_scores.append(results['onset'])
                offset_scores.append(results['offset'])

                with open(full_eval_path, 'a') as full_file, open(onset_eval_path, 'a') as onset_file, open(offset_eval_path, 'a') as offset_file:
                    full_file.write(f"{predicted_file}, {results['full'][0]}, {results['full'][1]}, {results['full'][2]}\n")
                    onset_file.write(f"{predicted_file}, {results['onset'][0]}, {results['onset'][1]}, {results['onset'][2]}\n")
                    offset_file.write(f"{predicted_file}, {results['offset'][0]}, {results['offset'][1]}, {results['offset'][2]}\n")

            except ValueError as e:
                print("Error")
                pass

    def calculate_and_save_averages(scores, file_path):
        if scores:
            avg_precision = sum(score[0] for score in scores) / len(scores)
            avg_recall = sum(score[1] for score in scores) / len(scores)
            avg_f1 = sum(score[2] for score in scores) / len(scores)
            with open(file_path, 'w') as file:
                file.write(f"Average Precision, Average Recall, Average F1 Score\n")
                file.write(f"{avg_precision}, {avg_recall}, {avg_f1}\n")

    calculate_and_save_averages(full_scores, full_avg_path)
    calculate_and_save_averages(onset_scores, onset_avg_path)
    calculate_and_save_averages(offset_scores, offset_avg_path)

predicted_folder = "OMAPS2/evaluation/skipping-the-frame-level"
truth_folder = "OMAPS2/complete/text"

evaluate_folders(predicted_folder, truth_folder, predicted_folder)


Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error


TypeError: absolute_error() missing 2 required positional arguments: 'reference_timestamps' and 'estimated_timestamps'

In [ ]:
predicted_intervals, predicted_pitches = read_note_data_from_npy("OMAPS2/evaluation/attack-delay/01_01-transcription.npy")
truth_intervals, truth_pitches = read_note_data_from_text("OMAPS2/complete/text/01_01.txt")

print("Predicted Intervals:", predicted_intervals)
print("Predicted Pitches:", predicted_pitches)
print("Truth Intervals:", truth_intervals)
print("Truth Pitches:", truth_pitches)

FileNotFoundError: [Errno 2] No such file or directory: 'OMAPS2/evaluation/attack-delay/01_01-transcription.npy'